In [2]:
import os
import openai
import requests
import json
import sseclient
import pandas as pd
import json

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
docs_chunks_path = "create_chunks/docs_chunks.json"
with open(docs_chunks_path, 'r', encoding="utf8") as f:
    docs_chunks = json.load(f)

docs_chunks[:2]

[{'header': ['Alert'],
  'link': '/docs/alert',
  'content': "A small box to quickly grab the user's attention and communicate a brief message.\n\n\nAn alert is an element that notifies the user of something important that is not user initiated. If you need to alert the user because of something they did, consider using a toast notification or an inline message. Instead, an alert is used to notify the user of something unrelated to what they just did. Examples of this might be announcing a new feature within the product, or alerting the user of scheduled server maintenance and downtime."},
 {'header': ['Alert', 'Appearance'],
  'link': '/docs/alert#appearance',
  'content': 'The alert message must be concise, we recommend no more than 256 characters. If you feel the need to explain in further detail, you may use an optional hyperlink at the end of the alert message to explain more on a new page. You may also include an optional close icon (close icon is required for sticky alerts).'}]

In [5]:
def get_embeddings(input_content, model="text-embedding-ada-002"):
    response = openai.Embedding.create(
        model=model,
        input=input_content
    )
    return response

In [15]:
embeddings_df = pd.read_parquet("data/embeddings_df.parquet", engine='fastparquet')
embeddings_df.head()

,chunk,object,embedding_object,index,embedding,prompt_tokens,total_tokens
level_0,,,,,,,
0,"{'header': ['Alert'], 'link': '/docs/alert', '...",list,embedding,0,"[-0.0216156505048275, 0.03179503604769707, 0.0...",122,122
1,"{'header': ['Alert', 'Appearance'], 'link': '/...",list,embedding,0,"[-0.002292848890647292, 0.04747709631919861, 0...",88,88


In [27]:
embeddings_df.loc[0, "chunk"]

{'header': ['Alert'],
 'link': '/docs/alert',
 'content': "A small box to quickly grab the user's attention and communicate a brief message.\n\n\nAn alert is an element that notifies the user of something important that is not user initiated. If you need to alert the user because of something they did, consider using a toast notification or an inline message. Instead, an alert is used to notify the user of something unrelated to what they just did. Examples of this might be announcing a new feature within the product, or alerting the user of scheduled server maintenance and downtime."}

In [19]:
sample_chunk = {
    "header": [
      "Alert"
    ],
    "link": "/docs/alert",
    "content": "A small box to quickly grab the user's attention and communicate a brief message.\n\n\nAn alert is an element that notifies the user of something important that is not user initiated. If you need to alert the user because of something they did, consider using a toast notification or an inline message. Instead, an alert is used to notify the user of something unrelated to what they just did. Examples of this might be announcing a new feature within the product, or alerting the user of scheduled server maintenance and downtime."
  }

In [25]:
embeddings_df["chunk" == sample_chunk]

KeyError: False

In [14]:
embeddings = {}
for chunk in docs_chunks:
    

['Alert']
['Alert', 'Appearance']
['Alert', 'Appearance', 'Informational']
['Alert', 'Appearance', 'Positive']
['Alert', 'Appearance', 'Warning']
['Alert', 'Appearance', 'Negative']
['Alert', 'Appearance', 'Multilines']
['Alert', 'Placement']
['Alert', 'Placement', 'Inline']
['Alert', 'Placement', 'Sticky']
['Alert', 'Props']
['Anchor']
['Anchor', 'Usage']
['Anchor', 'Usage', '`as` prop']
['Anchor', 'Props']
['Anchor', 'Related components']
['Avatar']
['Avatar', 'Appearance']
['Avatar', 'Appearance', 'Sizes']
['Avatar', 'Appearance', 'Initials']
['Avatar', 'Appearance', 'Icon']
['Avatar', 'Appearance', 'Picture']
['Avatar', 'Appearance', 'Status indicators']
['Avatar', 'Avatar group']
['Avatar', 'Avatar group', 'Stacked']
['Avatar', 'Avatar group', 'Animated stack']
['Avatar', 'Avatar group', 'Overflow']
['Avatar', 'Props']
['Badge']
['Badge', 'Variants']
['Badge', 'Variants', 'Soft backgrounds']
['Badge', 'Variants', 'Statuses']
['Badge', 'Usage']
['Badge', 'Usage', 'Label']
['Badge',